In [ ]:
import numpy as np
import sys
sys.path.append('../')
from cycada.data.data_loader import get_dataset
import PIL.Image as Image
from os.path import join
import os

In [ ]:
from torchvision import datasets

mnist_dataset = datasets.MNIST('../x/jhoffman/mnist/', train=True, transform=None, 
                               target_transform=None, download=True)
svhn_dataset = datasets.SVHN('../x/jhoffman/svhn/', split='train', transform=None, 
                             target_transform=None, download=True)

In [ ]:
import os
from PIL import Image

def process_dataset(gt_folder, img_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    images_output_folder = os.path.join(output_folder, 'images')
    if not os.path.exists(images_output_folder):
        os.makedirs(images_output_folder)

    labels_file = os.path.join(output_folder, 'labels.txt')
    with open(labels_file, 'w') as labels_f:
        img_index = 0

        for gt_file in os.listdir(gt_folder):
            gt_path = os.path.join(gt_folder, gt_file)
            img_name = gt_file.replace('gt_', '').replace('.txt', '.jpg')
            img_path = os.path.join(img_folder, img_name)

            if os.path.exists(img_path):
                img = Image.open(img_path)
                with open(gt_path, 'r') as f:
                    for line in f:
                        parts = line.strip().split(',')
                        if len(parts) == 9:
                            coordinates = list(map(int, parts[:8]))
                            label = parts[8]

                            # Define bounding box
                            left = min(coordinates[0::2])
                            top = min(coordinates[1::2])
                            right = max(coordinates[0::2])
                            bottom = max(coordinates[1::2])

                            # Crop and save image
                            cropped_img = img.crop((left, top, right, bottom))
                            cropped_img_path = os.path.join(images_output_folder, f'{img_index}.png')
                            cropped_img.save(cropped_img_path)

                            # Write label
                            labels_f.write(f'{img_index} {label}\n')
                            img_index += 1

# Process each dataset
process_dataset('../2013/Challenge2_Training_Task1_GT', '../2013/Challenge2_Training_Task12_Images', '../FY/Train/TrainA')
process_dataset('../2013/Challenge2_Test_Task12_Images', '../2013/Challenge2_Test_Task12_Images 2', '../FY/Test/TestA')
process_dataset('../2015/ch4_training_localization_transcription_gt', '../2015/ch4_training_images', '../FY/Train/TrainB')
process_dataset('../2015/Challenge4_Test_Task1_GT', '../2015/ch4_test_images', '../FY/Test/TestB')


In [ ]:
import os
from PIL import Image

def process_dataset(gt_folder, img_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    images_output_folder = os.path.join(output_folder, 'images')
    if not os.path.exists(images_output_folder):
        os.makedirs(images_output_folder)

    labels_file = os.path.join(output_folder, 'labels.txt')
    with open(labels_file, 'w') as labels_f:
        img_index = 0

        for gt_file in os.listdir(gt_folder):
            gt_path = os.path.join(gt_folder, gt_file)
            img_name = gt_file.replace('gt_', '').replace('.txt', '.jpg')
            img_path = os.path.join(img_folder, img_name)

            if os.path.exists(img_path):
                img = Image.open(img_path)
                try:
                    with open(gt_path, 'r', encoding='utf-8') as f:
                        lines = f.readlines()
                except UnicodeDecodeError:
                    with open(gt_path, 'r', encoding='ISO-8859-1') as f:
                        lines = f.readlines()

                for line in lines:
                    parts = line.strip().split(',')
                    if len(parts) == 9:
                        coordinates = list(map(int, parts[:8]))
                        label = parts[8]

                        # Define bounding box
                        left = min(coordinates[0::2])
                        top = min(coordinates[1::2])
                        right = max(coordinates[0::2])
                        bottom = max(coordinates[1::2])

                        # Crop and save image
                        cropped_img = img.crop((left, top, right, bottom))
                        cropped_img_path = os.path.join(images_output_folder, f'{img_index}.png')
                        cropped_img.save(cropped_img_path)

                        # Write label
                        labels_f.write(f'{img_index} {label}\n')
                        img_index += 1

# Process each dataset
process_dataset('../2013/Challenge2_Training_Task1_GT', '../2013/Challenge2_Training_Task12_Images', '../FY/Train/TrainA')
process_dataset('../2013/Challenge2_Test_Task12_Images', '../2013/Challenge2_Test_Task12_Images 2', '../FY/Test/TestA')
process_dataset('../2015/ch4_training_localization_transcription_gt', '../2015/ch4_training_images', '../FY/Train/TrainB')
process_dataset('../2015/Challenge4_Test_Task1_GT', '../2015/ch4_test_images', '../FY/Test/TestB')



In [ ]:
import os
from PIL import Image

def load_vocabulary(file_path):
    with open(file_path, 'r') as file:
        vocabulary = [line.strip() for line in file]

    word_to_index = {word: index for index, word in enumerate(vocabulary)}
    word_to_index["###"] = len(word_to_index)  # Handle special cases

    return word_to_index

def word_to_label(word, word_to_index):
    return word_to_index.get(word, 1)  # Returns the index for "###" if the word is not found

def process_dataset(gt_folder, img_folder, output_folder, word_to_index, numeric_naming=False, diagonal_points=False):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    images_output_folder = os.path.join(output_folder, 'images')
    if not os.path.exists(images_output_folder):
        os.makedirs(images_output_folder)

    labels_file = os.path.join(output_folder, 'labels.txt')
    
    with open(labels_file, 'w') as labels_f:
        print(labels_f)
        img_index = 0

        for gt_file in os.listdir(gt_folder):
            gt_path = os.path.join(gt_folder, gt_file)
            print(gt_path)

            if numeric_naming:
                # Extract the numeric part for numeric naming convention
                numeric_part = gt_file.split('_')[1].split('.')[0]  # Extracts the number from '100.txt'
                img_name = f'{numeric_part}.jpg'  # Constructs the corresponding image filename
            else:
                # Extract the numeric part for 'img_X.jpg' naming convention
                numeric_part = gt_file.split('_')[2].split('.')[0]
                img_name = f'img_{numeric_part}.jpg'

            img_path = os.path.join(img_folder, img_name)
            print(img_path)

            if os.path.exists(img_path):
                img = Image.open(img_path)
                try:
                    with open(gt_path, 'r', encoding='utf-8') as f:
                        lines = f.readlines()
                except UnicodeDecodeError:
                    with open(gt_path, 'r', encoding='ISO-8859-1') as f:
                        lines = f.readlines()

                for line in lines:

                    if ',' in line:
                        parts = line.strip().split(',')
                        print (parts)
                    else:
                        parts = line.strip().split(' ')
                        print (parts)
                   
                    if ((len(parts) == 9) or (len(parts) == 5)):
                       
                        if (len(parts) == 5):
                            try:
                                coordinates = list(map(int, parts[:4]))
                                label_word = parts[4].upper()
                                print(label_word)
                                label_index = word_to_label(label_word, word_to_index)  # Convert word to label index 
                                print(label_index)

                                if (label_index == 87623):
                                    label_index=0
                                else:
                                    label_index=1
                            except ValueError:
                                continue
                
                            # If coordinates are diagonal points (x1, y1, x3, y3)
                            x1, y1, x3, y3 = coordinates[0], coordinates[1], coordinates[2], coordinates[3]
                            left, top = min(x1, x3), min(y1, y3)
                            right, bottom = max(x1, x3), max(y1, y3)
                        else:
                            try:
                                coordinates = list(map(int, parts[:8]))
                                label_word = parts[8]
                                print (label_word)
                                
                                label_index = word_to_label(label_word, word_to_index)  # Convert word to label index 
                                if (label_index == 87623):
                                    label_index=0
                                else:
                                    label_index=1


                            except ValueError:
                                continue
                            # Regular bounding box coordinates
                            left, top, right, bottom = min(coordinates[0::2]), min(coordinates[1::2]), max(coordinates[0::2]), max(coordinates[1::2])

                        # Define bounding box and crop image
                        cropped_img = img.crop((left, top, right, bottom))
                        cropped_img_path = os.path.join(images_output_folder, f'{img_index}.png')
                        cropped_img.save(cropped_img_path)
                        print(cropped_img_path)

                        # Write label index
                        labels_f.write(f'{img_index} {label_index}\n')
                        img_index += 1
# Load vocabulary
vocabulary_file = '../GenericVocabulary.txt'  # Replace with the path to your vocabulary file
word_to_index = load_vocabulary(vocabulary_file)

# Process each dataset
process_dataset('../2013/Challenge2_Training_Task1_GT', '../2013/Challenge2_Training_Task12_Images', '../FY/train/trainA', word_to_index, numeric_naming=True, diagonal_points=True)
process_dataset('../2013/Challenge2_Test_Task1_GT', '../2013/Challenge2_Test_Task12_Images', '../FY/test/testA',word_to_index, numeric_naming=False, diagonal_points=True)
process_dataset('../2015/ch4_training_localization_transcription_gt', '../2015/ch4_training_images', '../FY/train/trainB',word_to_index, numeric_naming=False, diagonal_points=False)
process_dataset('../2015/Challenge4_Test_Task1_GT', '../2015/ch4_test_images', '../FY/test/testB',word_to_index, numeric_naming=False,diagonal_points=False)


In [ ]:
import os
from PIL import Image



# Load vocabulary and process datasets, using the numeric_naming parameter as needed


In [ ]:
outdir = '../x/jhoffman/cyclegan_data/svhn2mnist'
# SVHN->MNIST: convert to 32x32x3 images

os.makedirs(join(outdir, 'trainB/images'), exist_ok=True)
with open(join(outdir, 'trainB/labels.txt'), 'w') as label_file:
    for i in range(len(mnist_dataset)):
        img = Image.fromarray(mnist_dataset.train_data[i].numpy())
        img = img.resize([32,32])
        img = img.convert('RGB')
        img.save(join(outdir, 'trainB/images', '{:d}.png'.format(i)))
        label_file.write('{:d} {:d}\n'.format(i, mnist_dataset.train_labels[i]))
     

os.makedirs(join(outdir, 'trainA/images'), exist_ok=True)
svhn_labels = svhn_dataset.labels.flatten()
with open(join(outdir, 'trainA/labels.txt'), 'w') as label_file:
    for i in range(len(svhn_dataset)):
        img = Image.fromarray(svhn_dataset.data[i].transpose(1,2,0))
        img.save(join(outdir, 'trainA/images', '{:d}.png'.format(i)))
        label_file.write('{:d} {:d}\n'.format(i, svhn_labels[i]))

In [58]:
from PIL import Image
import os

def process_dataset(data_folder, output_folder, target_size=(64, 64), convert_to_rgb=False):
    images_folder = os.path.join(data_folder, 'images')
    labels_file = os.path.join(data_folder, 'labels.txt')

    os.makedirs(output_folder, exist_ok=True)
    os.makedirs(os.path.join(output_folder, 'images'), exist_ok=True)

    with open(labels_file, 'r') as labels_f, open(os.path.join(output_folder, 'labels.txt'), 'w') as out_labels_f:
        for line in labels_f:
            idx, label = line.strip().split()
            img_path = os.path.join(images_folder, f'{idx}.png')
            img = Image.open(img_path)

            # Resize every image to the target size
            img = img.resize(target_size)

            if convert_to_rgb:
                img = img.convert('RGB')

            img.save(os.path.join(output_folder, 'images', f'{idx}.png'))
            out_labels_f.write(f'{idx} {label}\n')

# Example usage
base_dir = '../FY/train'
process_dataset(os.path.join(base_dir, 'trainA'), '../FY/cyclegan_data/ICDAR2013_2015/trainA', target_size=(128, 128), convert_to_rgb=True)
process_dataset(os.path.join(base_dir, 'trainB'), '../FY/cyclegan_data/ICDAR2013_2015/trainB', target_size=(128, 128), convert_to_rgb=True)

# Example usage
base_dir = '../FY/test'
process_dataset(os.path.join(base_dir, 'testA'), '../FY/cyclegan_data/ICDAR2013_2015/testA', target_size=(128, 128), convert_to_rgb=True)
process_dataset(os.path.join(base_dir, 'testB'), '../FY/cyclegan_data/ICDAR2013_2015/testB', target_size=(128, 128), convert_to_rgb=True)

In [63]:
import shutil
import os

source_folder = '../FY/cyclegan_data/ICDAR2013_2015/trainA'
destination_folder = '../FY/icdar2013/train'

# Copy the folder
shutil.copytree(source_folder, destination_folder)

print(f"Folder copied from {source_folder} to {destination_folder}")

source_folder = '../FY/cyclegan_data/ICDAR2013_2015/trainB'
destination_folder = '../FY/icdar2015/train'

# Copy the folder
shutil.copytree(source_folder, destination_folder)

print(f"Folder copied from {source_folder} to {destination_folder}")

source_folder = '../FY/cyclegan_data/ICDAR2013_2015/testA'
destination_folder = '../FY/icdar2013/test'

# Copy the folder
shutil.copytree(source_folder, destination_folder)

print(f"Folder copied from {source_folder} to {destination_folder}")

source_folder = '../FY/cyclegan_data/ICDAR2013_2015/testB'
destination_folder = '../FY/icdar2015/test'

# Copy the folder
shutil.copytree(source_folder, destination_folder)

print(f"Folder copied from {source_folder} to {destination_folder}")


Folder copied from ../FY/cyclegan_data/ICDAR2013_2015/trainA to ../FY/icdar2013/train
Folder copied from ../FY/cyclegan_data/ICDAR2013_2015/trainB to ../FY/icdar2015/train
Folder copied from ../FY/cyclegan_data/ICDAR2013_2015/testA to ../FY/icdar2013/test
Folder copied from ../FY/cyclegan_data/ICDAR2013_2015/testB to ../FY/icdar2015/test


In [ ]:
# CUDA_VISIBLE_DEVICES=1 
# !python train.py --name experiment_name \ 
# --dataroot path_to_gta2cityscape --resize_or_crop=crop --loadSize=360 --fineSize=360 --identity 1.0 \
# --which_model_netD n_layers --n_layers_D 3

In [ ]:
%cd 

In [ ]:
import os

# Set the environment variable

!python ../cyclegan/pytorch-CycleGAN-and-pix2pix/train.py --name test_svhn2mnist \
--dataroot ../x/jhoffman/cyclegan_data/svhn2mnist/ --resize_or_crop=None \
--loadSize=32 --fineSize=32 --which_model_netD n_layers --n_layers_D 3 \
--no_flip --model cycle_gan --lambda_A 1 --lambda_B 1 --lambda_identity 1.0 --gpu_ids 0

In [ ]:
!ls

In [ ]:
!ls

In [ ]:
import sys
import os

print(sys.path)

In [ ]:
sys.path.insert(0, '/Users/pomvrp/Documents/NTU AI Courses/AI6121 computer vision/18 Nov Project/cycada_release/cycada')

In [ ]:
%cd '/Users/pomvrp/Documents/NTU AI Courses/AI6121 computer vision/18 Nov Project/cycada_release'
!ls

In [ ]:


import sys
import os
# A domain adaptive semantic segmentation model that is trained with the translated source data and evaluated over the target data.
! python cycada/scripts/train_adda.py --name ICDAR2013to2015-adaptive \
--dataroot ../../FY/cyclegan_data/ICDAR2013_2015/ --resize_or_crop=None \
--loadSize=32 --fineSize=32 --which_model_netD n_layers --n_layers_D 3 \
--no_flip --model LeNet --lambda_A 1 --lambda_B 1 --lambda_identity 1.0 --gpu_ids -1

In [ ]:
%cd ../

In [ ]:
import subprocess

# Define the path to your .sh file
script_path = "scripts/train_fcn_adda.sh"

# Run the shell script
subprocess.run(["bash", script_path])

In [ ]:
dirname = join(outdir, 'trainB')
for i in range(10):
    img = Image.open(join(dirname, '{:d}.png'.format(i)))

    plt.subplot(5,5,i+1)
    plt.imshow(img)
    plt.grid('off')
    plt.axis('off')

In [ ]:
dirname = join(outdir, 'trainA')
for i in range(10):
    img = Image.open(join(dirname, '{:d}.png'.format(i)))
    plt.subplot(5,5,i+1)
    plt.imshow(img)
    plt.grid('off')
    plt.axis('off')

In [ ]:
y = svhn_dataset.labels

In [ ]:
count,bins = np.histogram(y.squeeze(), bins=10)
plt.bar(range(10), count); plt.title('P(Y) for SVHN')

In [ ]:
min_num = min(count)
ind = np.zeros((10,min_num), dtype=int)
for i in np.unique(y):
    binary_ind = np.where(y.squeeze() == i)[0]
    np.random.shuffle(binary_ind)
    
    ind[i-1,:] = binary_ind[:min_num]

ind = ind.flatten()
np.random.shuffle(ind)
y_new = y[ind]

In [ ]:
count_new,_ = np.histogram(y_new, bins=10)
plt.bar(range(10), count_new); plt.title('P(Y) SVHN balanced')

In [ ]:
with open('/x/jhoffman/cyclegan_data/svhn2mnist/trainA/labels.txt', 'r') as f:
    data = f.read().splitlines()

parse = np.array([(int(x.split(' ')[0]), int(x.split(' ')[1])) for x in data])
d = dict(parse)

In [ ]:
import sys

# Print original sys.path
print("Original sys.path:", sys.path)

# Define the path you want to remove
path_to_remove = '..'

# Remove the path if it exists in sys.path
sys.path = [p for p in sys.path if p != path_to_remove]

# Print modified sys.path
print("Modified sys.path:", sys.path)
